# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [123]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [124]:
response = requests.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl') #puxa os dados da internet

data = response.json() #transforma os dados em .json
df = pd.DataFrame(data['items']) #armazena os dados em um dataframe

df.head(3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,57874,Município,Cristal,4306056,RS,PVL02.004765/2022-12,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2000000.0,0,1,19/05/2022
1,63552,Município,Teixeira de Freitas,2931350,BA,PVL02.000567/2023-61,Deferido,17944.100713/2023-10,2023-05-16T19:25:14Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,15000000.0,0,1,23/05/2023
2,64284,Município,Rio Pardo,4315701,RS,PVL02.001302/2023-80,Deferido,17944.101864/2023-95,2023-05-03T19:36:02Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,21000000.0,0,1,08/05/2023


In [125]:
# 1) Seu código aqui

df_rj = df[df['uf'] == 'RJ'] #filtragem do df para Estado RJ

df_rj.head(3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
121,40740,Município,Petrópolis,3303906,RJ,PVL02.008447/2019-25,Deferido,17944.104583/2019-16,2020-07-31T13:38:52Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,20000000.0,0,0,06/08/2020
200,68463,Município,Rio de Janeiro,3304557,RJ,PVL02.000745/2024-34,Encaminhado à PGFN com manifestação técnica fa...,17944.001955/2024-11,2024-06-06T14:49:09Z,Operação contratual interna (com garantia da U...,Pró-Transporte,Instituição Financeira Nacional,Caixa Econômica Federal,Real,141430900.0,0,0,12/06/2024
480,56075,Município,Resende,3304201,RJ,PVL02.003504/2022-85,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,2160000.0,0,1,28/07/2022


In [126]:
# 2) Seu código aqui

status_counts = df['status'].value_counts() #contagem de cada atributo de 'status'
top3_status = status_counts.nlargest(3) #armazena os 3 maiores counts

top3_status

status
Deferido             2406
Deferido (PVL-IF)    1370
Arquivado             299
Name: count, dtype: int64

In [127]:
# 3) Seu código aqui

#descobri que tem como alterar o tipo do campo para data ao invés de transformar em string para depois manipular
#como sabemos que o formato de data no Brasil e nesse dataframe são DD/MM/YYYY, devemos usar dayfirst = True, para que não haja equívoco
#caso não seja especificado, ainda assim ficará correto, mas haverá um aviso de erro em potencial
df['data_status'] = pd.to_datetime(df['data_status'], dayfirst = True)

ano_status = df['data_status'].dt.year #esse comando extrai somente um dos 3 elementos day/month/year

ano_status #int


0       2022
1       2023
2       2023
3       2023
4       2024
        ... 
4995    2019
4996    2019
4997    2018
4998    2018
4999    2018
Name: data_status, Length: 5000, dtype: int32

In [128]:
# 4) Seu código aqui

#mesmo procedimento que o item 2, porém com as contagens de ano_status
ano_status_counts = ano_status.value_counts()
top3_ano_status = ano_status_counts.nlargest(3)

top3_ano_status

data_status
2023    515
2022    432
2019    322
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [129]:
#1) Seu código aqui

def filtro_uf_tipo(uf: str, tipo_interessado: str):
    
    response = requests.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl')
    data = response.json()
    df = pd.DataFrame(data['items'])

    #filtro para UF e tipo interessado aplicado a partir dos parametros da função
    filtro = (df['uf'] == uf) & (df['tipo_interessado'] == tipo_interessado)
    df_filtrado = df[filtro]

    return df_filtrado


In [130]:
# 2) Seu código aqui

uf = 'MG'
tipo_interessado = 'Estado'

df_filtrado = filtro_uf_tipo(uf, tipo_interessado)

df_arquivado = df_filtrado[df_filtrado['status'] == 'Arquivado por decurso de prazo']
df_arquivado

#não há nenhum item cujo tipo_interessado = 'Estado' e uf = 'MG', com status 'Arquivado por decurso de prazo'

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status


In [131]:
# 2) Seu código aqui --> resolvido de maneira alternativa, onde tipo_interessado = 'Município'

uf = 'MG'
tipo_interessado = 'Município'

df_filtrado = filtro_uf_tipo(uf, tipo_interessado)

df_arquivado = df_filtrado[df_filtrado['status'] == 'Arquivado por decurso de prazo']
df_arquivado.head()

#não há nenhum item cujo tipo_interessado = 'Estado' e uf = 'MG', com status 'Arquivado por decurso de prazo, somente tipo_interessado = 'Município'

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
36,26796,Município,Alvarenga,3102209,MG,PVL02.002291/2017-15,Arquivado por decurso de prazo,17944.102912/2017-14,2017-12-18T11:56:24Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,500000.0,0,0,20/02/2018
86,26654,Município,Machacalis,3138906,MG,PVL02.002257/2017-32,Arquivado por decurso de prazo,17944.102656/2017-65,2018-02-26T14:21:37Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,500000.0,0,0,15/08/2018
88,26819,Município,Divino,3122009,MG,PVL02.002329/2017-41,Arquivado por decurso de prazo,17944.103023/2017-74,2017-11-30T19:30:27Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,100000.0,0,0,19/02/2018
92,26878,Município,Muriaé,3143906,MG,PVL02.002355/2017-70,Arquivado por decurso de prazo,17944.103186/2017-57,2017-12-04T17:14:08Z,Operação contratual interna,"Aquisição de terrenos, imóveis e/ou construção...",Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,1500000.0,0,0,16/02/2018
522,26538,Município,Lambari,3137809,MG,PVL02.002088/2017-31,Arquivado por decurso de prazo,17944.102485/2017-74,2017-11-28T12:47:30Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,2000000.0,0,0,06/02/2018


In [132]:
# 3) Seu código aqui

#municipios - BA
uf = 'BA'
tipo_interessado = 'Município'

df_filtrado = filtro_uf_tipo(uf, tipo_interessado)

#para facilitar a visualização, restringi as colunas para um menor número
colunas_desejadas = ['tipo_interessado', 'interessado', 'uf', 'status']
df_desejado = df_filtrado[colunas_desejadas]

contagem_deferidos = df_desejado[df_desejado['status'] == 'Deferido'].value_counts()

municipio_deferidos_1 = contagem_deferidos.head(1) #top 1
municipio_deferidos_1

tipo_interessado  interessado  uf  status  
Município         Camaçari     BA  Deferido    4
Name: count, dtype: int64

In [133]:
# 4) Seu código aqui

uf = 'BA'
tipo_interessado = 'Estado'

df_filtrado = filtro_uf_tipo(uf, tipo_interessado)

df_filtrado.to_csv('./interessados_BA.csv')